In [18]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

# File paths
base_folder = r'F:\AlterG\Control\Data'
normative_traces_folder = r'F:\AlterG\NormativeTraces'
save_folder = r'F:\AlterG\IndividualKinematics'

In [19]:
# File paths
base_folder = r'F:\AlterG\Control\Data'
normative_traces_folder = r'F:\AlterG\NormativeTraces'
save_folder = r'F:\AlterG\IndividualKinematics'

# Step 1: Load Participant Data
def load_participant_data(participant_number):
    participant_folder = os.path.join(base_folder, f"{participant_number:02d}")
    kinematics_path = os.path.join(participant_folder, 'Kinematics', 'concatenated_strides.pkl')
    devices_data_path = os.path.join(participant_folder, 'Gait', 'DevicesData.mat')
    
    if os.path.exists(kinematics_path):
        with open(kinematics_path, 'rb') as f:
            participant_data = pickle.load(f)
            return participant_data
    elif os.path.exists(devices_data_path):
        try:
            mat_data = loadmat(devices_data_path)
            print("Successfully loaded data using `loadmat`.")
            return mat_data
        except NotImplementedError:
            print(f"Error: The file '{devices_data_path}' may be saved in a newer HDF5-compatible format.")
        except Exception as e:
            print(f"Failed to load '{devices_data_path}': {e}")
            return None
    else:
        print(f"No data found for Participant {participant_number}")
        return None


In [20]:
# Step 2: Load and Process Each Participant
def process_participant(participant_number, normative_directory, output_directory):
    participant_data = load_participant_data(participant_number)
    if participant_data is None:
        return

    corrected_data = {}
    for header, strides in participant_data.items():
        if header == 'time':  # Skip 'time' header as it is not used for correction
            continue
        try:
            normative_trace = load_saved_normative(normative_directory, header)
        except FileNotFoundError as e:
            print(e)
            continue

        corrected_strides = [correct_stride(stride, normative_trace) for stride in strides]
        corrected_data[header] = corrected_strides

        # Debugging: Print or plot to verify normative trace and strides
        print(f"Processing header: {header}")
        plt.figure()
        plt.plot(normative_trace, label='Normative Trace', color='b')
        plt.plot(strides[0], label='First Original Stride', color='r', alpha=0.5)
        plt.legend()
        plt.title(f"Normative vs Original Stride for Header: {header}")
        plt.show()

        # Visualization
        visualize_strides(strides, corrected_strides, participant_number)

    # Save corrected data
    save_corrected_data(output_directory, participant_number, corrected_data)


In [21]:
# Step 3: Load Normative Trace
def load_saved_normative(normative_directory, header):
    normative_file = os.path.join(normative_directory, f"{header}.pkl")
    if os.path.exists(normative_file):
        with open(normative_file, 'rb') as f:
            return pickle.load(f)
    else:
        raise FileNotFoundError(f"Normative trace for header {header} not found.")




In [22]:
# Step 4: Correct Individual Strides
def correct_stride(stride, normative_trace):
    # Correlation-based flipping
    correlation = np.corrcoef(stride, normative_trace)[0, 1]
    flipped_correlation = np.corrcoef(-stride, normative_trace)[0, 1]
    if flipped_correlation > correlation:
        stride = -stride

    # Offset correction (demean if necessary)
    stride_mean = np.mean(stride)
    normative_mean = np.mean(normative_trace)
    if abs(stride_mean - normative_mean) > 0.1 * normative_mean:  # threshold at 10%
        stride = stride - stride_mean + normative_mean

    return stride

In [23]:
# Step 5: Visualization
def visualize_strides(original_strides, corrected_strides, participant_id):
    plt.figure(figsize=(12, 8))

    # Original strides
    plt.subplot(2, 1, 1)
    for stride in original_strides:
        plt.plot(stride, alpha=0.5)
    plt.title(f"Participant {participant_id} - Original Strides")
    plt.xlabel("Time Points")
    plt.ylabel("Stride Value")

    # Corrected strides
    plt.subplot(2, 1, 2)
    for stride in corrected_strides:
        plt.plot(stride, alpha=0.5)
    plt.title(f"Participant {participant_id} - Corrected Strides")
    plt.xlabel("Time Points")
    plt.ylabel("Stride Value")

    plt.tight_layout()
    plt.show()

In [24]:
# Step 6: Save Corrected Data
def save_corrected_data(output_directory, participant_id, corrected_data):
    output_file = os.path.join(output_directory, f"{participant_id:02d}_corrected.pkl")
    with open(output_file, 'wb') as f:
        pickle.dump(corrected_data, f)




In [25]:
# Main Processing Pipeline
def process_stride_data(participant_numbers, normative_directory, output_directory):
    for participant_number in participant_numbers:
        process_participant(participant_number, normative_directory, output_directory)

# Notebook Call for Testing
participant_numbers = [1]  # Testing with the first participant only
normative_directory = normative_traces_folder
output_directory = save_folder

process_stride_data(participant_numbers, normative_directory, output_directory)

Normative trace for header pelvis_tilt not found.
Normative trace for header pelvis_list not found.
Normative trace for header pelvis_rotation not found.
Normative trace for header pelvis_tx not found.
Normative trace for header pelvis_ty not found.
Normative trace for header pelvis_tz not found.
Normative trace for header lumbar_extension not found.
Normative trace for header lumbar_bending not found.
Normative trace for header lumbar_rotation not found.
Normative trace for header hip_flexion_r not found.
Normative trace for header hip_adduction_r not found.
Normative trace for header hip_rotation_r not found.
Normative trace for header knee_angle_r not found.
Normative trace for header ankle_angle_r not found.
Normative trace for header subtalar_angle_r not found.
Normative trace for header mtp_angle_r not found.
Normative trace for header hip_flexion_l not found.
Normative trace for header hip_adduction_l not found.
Normative trace for header hip_rotation_l not found.
Normative trac

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\AlterG\\IndividualKinematics\\01_corrected.pkl'